In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

"""
softmax函数的作用
1. 将所有的神经网络的输出值变为正数。 exp{p(yhat)}
2. 多个输出归一化到合法的概率分布。 exp{p(yhat)} / sum(exp{yhat})
"""
def my_softmax(X):
    """
    自己定义的softmax
    """
    X_exp = X.exp()
    X_exp_sum = X_exp.sum(axis=1, keepdims=True)
    return X_exp / X_exp_sum

x_input = torch.randn(1, 5)
print('x_input:\n',x_input)
y_target = torch.tensor([1])
print('y_target\n',y_target)
"""
计算输入softmax，此时可以看到每一行加到一起结果都是1
奇怪的是这些函数都需要用类似函数指针的方法使用
nn.Softmax()
nn.CrossEntropyLoss()
"""
softmax_func = nn.Softmax(dim=1)
soft_output = softmax_func(x_input)
print('soft_output:\n', soft_output)
my_softmax_output = my_softmax(x_input)
print('my_softmax_output:\n', my_softmax_output)
"""
注意:
    x_input的行数跟y_target的长度一样。
    y_target的元素的值要小于x_input的列数。
    可以一次计算多个sample的cross entropy。
    不需要对x_input做softmax再传入。
"""
crossentropyloss = nn.CrossEntropyLoss()
crossentropyloss_output = crossentropyloss(x_input, y_target)
print(crossentropyloss_output)
x_input2 = torch.randn(2, 5)
# y_target2 = torch.tensor([1, 7]) - error, 7 > 4
y_target2 = torch.tensor([1, 4])

crossentropyloss_output2 = crossentropyloss(x_input2, y_target2)
print(crossentropyloss_output2)

In [ ]:
# 生成一些两类假数据
n_data = torch.ones(100, 2)
# normal 这个函数应该初始化时经常用到
x0 = torch.normal(2*n_data, 1)      # class0 x data (tensor), shape=(100, 2)
y0 = torch.zeros(100)               # class0 y data (tensor), shape=(100, 1)
x1 = torch.normal(-2*n_data, 1)     # class1 x data (tensor), shape=(100, 2)
y1 = torch.ones(100)                # class1 y data (tensor), shape=(100, 1)
x = torch.cat((x0, x1), 0).type(torch.FloatTensor)  # shape (200, 2) FloatTensor = 32-bit floating
y = torch.cat((y0, y1), 0).type(torch.LongTensor)    # shape (200,) LongTensor = 64-bit integer

# print(n_data)
# print(x0)
# print(x1)
plt.scatter(x.data.numpy()[:, 0], x.data.numpy()[:, 1], c=y.data.numpy(), s=100, lw=0, cmap='RdYlGn')
plt.show()

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.out = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        """
        输出两个浮点数。
        这里没有在网络中做softmax，而是出去以后做的。
        """
        x = F.relu(self.hidden(x))     
        x = self.out(x)
        return x

net = Net(n_feature=2, n_hidden=10, n_output=2)     # define the network
print(net) 
optimizer = torch.optim.SGD(net.parameters(), lr=0.02)
# 分类问题都使用CrossEntropyLoss。
loss_func = torch.nn.CrossEntropyLoss()  # the target label is NOT an one-hotted
plt.ion()
for t in range(100):
    out = net(x)
    loss = loss_func(out, y) # must be (1. nn output, 2. target), the target label is NOT one-hotted
    optimizer.zero_grad()   
    loss.backward()         
    optimizer.step()
    if t % 2 == 0:
        plt.cla()
        prediction = torch.max(out, 1)[1]
        pred_y = prediction.data.numpy()
        target_y = y.data.numpy()
        plt.scatter(x.data.numpy()[:, 0], x.data.numpy()[:, 1], c=pred_y, s=100, lw=0, cmap='RdYlGn')
        accuracy = float((pred_y == target_y).astype(int).sum()) / float(target_y.size)
        plt.text(1.5, -4, 'Accuracy=%.2f' % accuracy, fontdict={'size': 20, 'color':  'red'})
        plt.pause(0.1)
plt.ioff()
plt.show()